# Pré-Processamento de texto

## Importações

In [1]:
# Imports
import sys
import os
import pandas as pd
import unicodedata
import nltk
from nltk.util import ngrams
import pickle
import re
from nltk import tokenize # Tokenização
from nltk.corpus import stopwords # Stopwords
from nltk.stem.lancaster import LancasterStemmer
from collections import Counter

## Imports e definição de funções

In [2]:
# Inicializa o stemmer
stemmer = LancasterStemmer() 
# Stop words para o Português
portuguese_stops = set(stopwords.words('portuguese'))
portuguese_stops.update('é')
# Lista de palavras para remover com regex
include_regex = [r"^[A-Za-zçãàáâéêíóôõúüÂÃÁÀÉÊÍÓÔÕÚÜÇ'º°ª\.]*$",r"^[0-9\-/\\\.]*$"]
#include_regex = [r"^[A-Za-zçãàáâéêíóôõúüÂÃÁÀÉÊÍÓÔÕÚÜÇ'º°ª\.]*$"]
LIMITE_TAMANHO_PALAVRA = 3
# Dicionátio que armazena as pontuações a serem removidas
tbl = {} 
for code in [i for i in range(sys.maxunicode) if unicodedata.category(chr(i)).startswith('P')]:
    tbl[code] = ' '
# Função para remover pontuações das sentenças
def remove_punctuation(text): 
    return text.translate(tbl)
# Função para salvar uma lista em um binário
def save_list_to_file(list_to_save,file_name):
    with open(file_name, 'wb') as f:
        pickle.dump(list_to_save, f)
# Função para realizar o pré-processamento dos conteúdos dos arquivos
def create_doc(txt): 
    docs = []
    ngrs = []
    ret = {}
    with open(os.path.join('texto_extraido',txt),'r') as f:
        content = remove_punctuation(f.read())
        words = tokenize.word_tokenize(content, language='portuguese')     
        # Remove stopwords
        words = [word for word in words if word.lower() not in portuguese_stops] 
        # Stem de cada palavra, converte para minúsculo 
        words = [stemmer.stem(w.lower()) for w in words] 
        # Remove palavras menores que um limite
        words = [word for word in words if len(word) > LIMITE_TAMANHO_PALAVRA] 
        # Remove regex words
        words_aux = []
        for word in words:
            add = False
            for regex_exp in include_regex:
                if re.compile(regex_exp).match(word):
                    add = True
            if add:
                words_aux.append(word)
            #else:
            #    print ('removerndo regex: ',word)
        words = words_aux
        for ngr in ngrams(words,2):
            ngrs.append(ngr)
        ret['ngrams'] = ngrs
        # Busca resultados e adiciona nos docs
        reg = df.loc[df['Nome do Arquivo'].str.contains(os.path.splitext(txt)[0])] # Busca o resultado do arquivo 
        cats = []
        for cat in categories:
            if not reg[cat].isnull().values[0]:
                # Uma lista de tuplas com as palavras das sentenças e o nome da categoria
                cats.append(cat)
        docs.append((ngrs, cats, txt))
    ret['docs'] = docs
    return ret

## Importanto os documentos e planilha e pré-processando o texto

In [3]:
# Importando a planilha com os dados de treinamento no formato CSV
df = pd.read_csv('resultwex2.csv') 
# Gera uma lista com todas as categorias de classificação a partir do CSV
categories = list(df.columns[2:13])
# Lista de todas as palavras de todos os documentos
all_ngrams = []
# Uma lista de tuplas com as palavras das sentenças e o nome da categoria
docs = [] 
ngr = []
list_txt = os.listdir("texto_extraido")
count = 0
LIMITE = -1
for txt in list_txt:
    count += 1
    ret = create_doc(txt)
    docs.extend(ret['docs'])
    all_ngrams.extend(ret['ngrams'])
    if count % 100 == 0:
        print (count,' arquivos processados. Último processado: ',txt)
    if LIMITE != -1 and count == LIMITE:
        break
print (count,' arquivos processados. Último processado: ',txt)
print('\nForam geradas ',len(docs),' combinações e ',len(all_ngrams),' palavras')
print ('\n',len(categories),'Categorias: ',categories)
print ('\n',len(docs),'Docs (primeiro): ',docs[:1])
print ('\n',len(all_ngrams),' N-Grams: ',all_ngrams[:100])

100  arquivos processados. Último processado:  148494751_1_OCR.txt
200  arquivos processados. Último processado:  147672969_1_OCR.txt
300  arquivos processados. Último processado:  146626769_1_OCR.txt
400  arquivos processados. Último processado:  146594538_1_OCR.txt
500  arquivos processados. Último processado:  143667061_1_OCR.txt
600  arquivos processados. Último processado:  148623718_1_OCR.txt
700  arquivos processados. Último processado:  146509336_1_OCR.txt
800  arquivos processados. Último processado:  147130806_1_OCR.txt
900  arquivos processados. Último processado:  148796460_1_OCR.txt
1000  arquivos processados. Último processado:  146882472_1_OCR.txt
1100  arquivos processados. Último processado:  146660219_1_OCR.txt
1200  arquivos processados. Último processado:  146437513_1_OCR.txt
1300  arquivos processados. Último processado:  147726641_1_OCR.txt
1400  arquivos processados. Último processado:  150639780_1_OCR.txt
1500  arquivos processados. Último processado:  147748486

## Salvando os arquivos para uso posterior em outro programa

In [4]:
# Store data (serialize)
save_list_to_file(categories,'categories.pickle')
save_list_to_file(all_ngrams,'all_ngrams.pickle')
save_list_to_file(docs,'docs.pickle')
print ('Arquivos salvos com sucesso!')

Arquivos salvos com sucesso!


## Fim